In [1]:
import cvxpy
from matplotlib import pyplot as plt
import numpy as np
from src.components.synthetic_house import SyntheticHouse
from src.components.synthetic_microgrid import SyntheticMicrogrid
from src.environments.simple_microgrid import SimpleMicrogrid

from src.components.battery import Battery, BatteryParameters
from src.utils.tools import set_all_seeds, load_config

/home/roberto.guillen/.conda/envs/bcte/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Microgrid

## Functions

In [2]:
def solver(house: SyntheticHouse, n: int = 24):

    battery = cvxpy.Variable(n+1)
    action = cvxpy.Variable(n)
    consumption = cvxpy.Variable(n)

    constraints = []

    # Battery
        # Starts in 0.1
    constraints.append(battery[0] == house.battery.soc_min)
        # Max and min batteries
    for i in range(n+1):
        constraints.append(battery[i] <= house.battery.soc_max)
        constraints.append(battery[i] >= house.battery.soc_min)


    # Action / Batteryn't

    for i in range(n):
        constraints.append(action[i] <= 1)
        constraints.append(action[i] >= -1)


    # Transition
    obj = 0

    for i in range(n):
        
        constraints.append(action[i] <= house.battery.p_charge_max)
        constraints.append(action[i] <= house.battery.p_discharge_max)
        # Update battery
        constraints.append(battery[i+1] == battery[i] + action[i] * house.battery.efficiency)
        # Update net 
        constraints.append(consumption[i] == house.demand[i]-house.pv_gen[i] + action[i] * house.battery.efficiency)


        obj += cvxpy.maximum(consumption[i] * (house.price[i] + house.emission[i]),0) 
        obj += cvxpy.maximum(-consumption[i] * house.price[i] * house.grid_sell_rate,0)  


    objective = cvxpy.Minimize(obj)
    prob = cvxpy.Problem(objective, constraints)
    res = prob.solve()

    return res, battery.value, action.value

In [22]:
def get_all_actions(env: SimpleMicrogrid, mode : str = 'train') -> np.ndarray:
    # Set mode, train, eval, test
    env.mg.change_mode(mode)

    # Create arrays to hold score, battery SOC and Action for all houses
    rewards, battery_values, action_values = [],[],[]
    
    # Same for all houses
    for house in env.mg.houses:
        reward, batt, action = solver(house)
        
        rewards.append(reward)
        battery_values.append(batt)
        action_values.append(action)
    battery_values = np.array(battery_values)
    action_values = np.array(action_values)
    rewards = np.array(rewards)

    return rewards, battery_values, action_values
    # print("Mean", scores.mean(), "\n Scores",rewards)    

In [23]:
def loop_env(env: SimpleMicrogrid, action_values: np.ndarray, mode : str = 'train') -> np.ndarray:
    done = 0
    env.mg.change_mode(mode)

    env.reset()

    # Cycle the entire episode with already computed actions by solver
    while not done:
        time_step = env.mg.current_step
        _,_,done,_ = env.step(action_values[:,time_step])
        time_step += 1

    # Calculate Score with actions
    return env.mg.get_houses_metrics() # output price, emissions 

## Run this

In [24]:
set_all_seeds(0)
# create environment,m save array of houses
config = load_config("zero_mg")
env = SimpleMicrogrid(config=config['env'])

# Train
mode = 'train'
rewards, battery_values, action_values = get_all_actions(env, mode)
train_metrics = loop_env(env, action_values, mode)

# Eval
mode = 'eval'
rewards, battery_values, action_values = get_all_actions(env, mode)
eval_metrics = loop_env(env, action_values, mode)

# Test
mode = 'test'
rewards, battery_values, action_values = get_all_actions(env, mode)
test_metrics = loop_env(env, action_values, mode)


In [26]:
train_metrics, eval_metrics, test_metrics

((array([-0.09929886, -0.16271739, -0.04808466, -0.15198558,  0.03388161,
          0.03724827]),
  array([-0.23935303, -0.31534628, -0.13570712, -0.28165016, -0.0298125 ,
          0.00033534])),
 (array([-0.18073769, -0.13861451, -0.13705395,  0.01801452, -0.08770782,
          0.02134148]),
  array([-0.35173296, -0.27818922, -0.25554218, -0.02177083, -0.17006995,
         -0.02798345])),
 (array([-0.17577385, -0.25198997, -0.18895732,  0.04679109, -0.04659103,
          0.03022902,  0.00812369, -0.0376431 ,  0.06172703, -0.03975418]),
  array([-3.22765870e-01, -4.61615321e-01, -3.43667000e-01, -2.69293021e-04,
         -1.39544784e-01, -6.55449237e-03, -6.60324787e-02, -7.36377648e-02,
          8.35868715e-03, -7.45623736e-02])))

## Train

In [16]:
# Create arrays to hold score, battery SOC and Action for all houses
scores, battery_values, action_values = [],[],[]
 
# Same for all houses
for house in env.mg.houses:
    score, batt, action = solver(house)
    scores.append(score)
    battery_values.append(batt)
    action_values.append(action)
battery_values = np.array(battery_values)
action_values = np.array(action_values)
scores = np.array(scores)
print("Mean", scores.mean(), "\n Scores",scores)

metrics = loop_env(env, action_values)
metrics[0].mean()

Mean 1.0271841323703481 
 Scores [0.99668626 1.62052129 1.42979519 1.29279424 0.48693118 0.33637664]


-0.06636514094694872

## Eval

In [19]:
set_all_seeds(0)
env.mg.change_mode('eval')

# Create arrays to hold score, battery SOC and Action for all houses
scores, battery_values, action_values = [],[],[]
 

# Same for all houses
for house in env.mg.houses:
    score, batt, action = solver(house)
    scores.append(score)
    battery_values.append(batt)
    action_values.append(action)
battery_values = np.array(battery_values)
action_values = np.array(action_values)
scores = np.array(scores)
print("Mean", scores.mean(), "\n Scores",scores)

metrics = loop_env(env, action_values)
metrics[0].mean()

Mean 0.8785728684997188 
 Scores [1.21527128 1.15445464 1.63444597 0.31410258 0.60907768 0.34408506]


-0.08411612353662445

## Test

In [20]:
set_all_seeds(0)
env.mg.change_mode('test')

# Create arrays to hold score, battery SOC and Action for all houses
scores, battery_values, action_values = [],[],[]
 

# Same for all houses
for house in env.mg.houses:
    score, batt, action = solver(house)
    scores.append(score)
    battery_values.append(batt)
    action_values.append(action)
battery_values = np.array(battery_values)
action_values = np.array(action_values)
scores = np.array(scores)
print("Mean", scores.mean(), "\n Scores",scores)

metrics = loop_env(env, action_values)
metrics[0].mean()

Mean 1.655768645368282 
 Scores [2.52973792 4.27896408 3.16027049 0.24285223 0.84557628 0.60987541
 0.42697884 1.61554766 0.33510372 2.51277981]


-0.06044315681587431

# House

### Set env vars

In [47]:
set_all_seeds(0)

# Same env obs as test simple env

config = load_config("c_a2c")
config = config['train']

mg = SyntheticHouse(config=config['env'])
demand = mg.demand
pv = mg.pv_gen
price_s = mg.price
emission = mg.emission
price_b = price_s / 4

In [48]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
    
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

0.8200000000000001

### Adv battery calc (not working)

In [49]:
set_all_seeds(0)
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't
# for i in range(n):
#     constraints.append(action[i] <= 0.9)
#     constraints.append(action[i] >= -0.9)

# Transition
obj = 0

for i in range(n):
    # Max and min battery charge 
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery SOC
    # self.soc = self.soc + (p_charge * self.efficiency - p_discharge / self.efficiency) / self.capacity

    constraints.append(battery[i+1] == battery[i] + ((action[i] * battery_params["efficiency"])/battery_params["efficiency"])/battery_params["capacity"] )
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value, consumption.value, price_s

NameError: name 'n' is not defined

### Real dataset

In [ ]:
n = 24

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery
    constraints.append(battery[i+1] == battery[i] + action[i] * battery_params["efficiency"])
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

### Test Dataset

In [ ]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4

In [ ]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value